In [1]:
# INPUT:
# TODO: Build simple ui or just use terminal for inputs
# TODO: Allow for more in-depth customization in args (argparse)

# SCORING:
# TODO: Score distance using city block method (will be in the nitty gritty area)
# TODO: Score avg distance across trials (will be in the nitty gritty area)
# TODO: The PRACT subset only has 3 instead of 4 inputs! (fix this)

# OUTPUT:
# TODO: Display more in-depth details further along (horizontal)
# TODO: Display avg distance from correct block (incorrect answers) in the 
# TODO: Print LOGS stating what is being done

# FINAL:
# TODO: Migrate .ipynb to .py format and finish syncing script

In [143]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plot
import os
import logging as log
import shutil

class Util:
    def __init__(self) -> None:
        self.currentDir = os.getcwd()

    def findFile(self, testName=str):

        # Search for a txt file in the current directory
        files_found = []
        for file in os.listdir(self.currentDir):
            if file.endswith('.txt') and testName in file:
                files_found.append(file)

        if len(files_found) == 0:
            print(f'No TXT file named {testName} found in current directory')
            return None
        elif len(files_found) == 1:
            # If a TXT file is found, read it into a Pandas dataframe
            df = pd.read_csv(os.path.join(self.currentDir, files_found[0]), delimiter='\t')
            print(f'Successfully read file: {files_found[0]}')
            return df
        else:
            print(f'{len(files_found)} files with the name {testName} found in current directory:')
            for i, file in enumerate(files_found):
                print(f'{i+1}. {file}')
            selection = input('Please select the file you want to read (enter number): ')
            try:
                selection = int(selection)
            except:
                print('Invalid input, please enter a number.')
                return None
            if selection > 0 and selection <= len(files_found):
                df = pd.read_csv(os.path.join(currentDir, files_found[selection-1]), delimiter='\t')
                print(f'Successfully read file: {files_found[selection-1]}')
                return df
            else:
                print(f'Invalid selection: {selection}. Please enter a number between 1 and {len(files_found)}.')
                return None

    def getTrialType(self, df=pd.DataFrame, trialLoc=4):
        
        # List different types of trials in the dataframe
        trialTypes = df.iloc[:, trialLoc].unique()
        
        return trialTypes
    
    def subsetByTrial(self, df=pd.DataFrame, trialTypes=np.ndarray, trialLoc=4):
        
        # Create a list to hold the subsetted dataframes
        subsetList = []
        
        # Subset data based on trial number
        for trial in trialTypes:
            # Create a new dataframe for each trial type
            trialDf = df[df.iloc[:, trialLoc] == trial]
            
            # Add each dataframe to a list
            subsetList.append(trialDf)
        
        # Return the list of dataframes
        return subsetList

    def makeScoredDir(self):

        # Create a new directory for the scored files
        newDir = os.path.join(self.currentDir, 'scored')
        if not os.path.exists(newDir):
            os.mkdir(newDir)
            print(f'Successfully created directory: {newDir}')
        else:
            print(f'Directory {newDir} already exists')
        
        # Move all files with 'scores' in their name to the new scored directory
        for file in os.listdir(self.currentDir):
            if 'scores' in file:
                filePath = os.path.join(self.currentDir, file)
                shutil.move(filePath, newDir)
                print(f'Successfully moved {file} to {newDir}')

class Main(Util):
    def __init__(self) -> None:
        super().__init__()


myUtil = Util()


In [134]:
def fsScore():
    # grab test document
    fs = myUtil.findFile('FiguralSpeed')

    # determine all unique trials
    trials = myUtil.getTrialType(fs)

    # subset data by trial
    trialDfs = myUtil.subsetByTrial(fs, trials)

    # create a list to store the results
    trialData = []

    # score each trial and add results to the list
    for trial in trialDfs:
        trialType = trial.iloc[0, 4]
        pc = trial.iloc[:, 13].mean()
        responseTimes = trial.iloc[:, 14].mean()
        responseCorrect = trial[trial.iloc[:, 13] == 1].iloc[:, 14].mean()
        responseIncorrect = trial[trial.iloc[:, 13] == 0].iloc[:, 14].mean()
        medianResponseTime = trial.iloc[:, 14].median()
        medianResponseCorrect = trial[trial.iloc[:, 13] == 1].iloc[:, 14].median()
        medianResponseIncorrect = trial[trial.iloc[:, 13] == 0].iloc[:, 14].median()

        trialData.append({
            'Trial Type': trialType,
            'Proportion Correct': pc,
            'Total Inputs': trial.iloc[:, 13].count(),
            'Average Response Time': responseTimes,
            'Average Response Time (Correct)': responseCorrect,
            'Average Response Time (Incorrect)': responseIncorrect,
            'Median Response Time': medianResponseTime,
            'Median Response Time (Correct)': medianResponseCorrect,
            'Median Response Time (Incorrect)': medianResponseIncorrect
        })

    # create a data frame from the results list
    df = pd.DataFrame(trialData)

    # save the data frame to a CSV file
    df.to_csv('fs_scores.csv', float_format='%.2f', index=False)

    # print the data frame
    print(df)

fsScore()


Successfully read file: 1001_FiguralSpeed.txt
  Trial Type  Proportion Correct  Total Inputs  Average Response Time  \
0    PRACT              0.948718            39            2245.512821   
1    TRIAL1             0.950000            40            2837.050000   
2    TRIAL2             0.900000            40            2558.875000   

   Average Response Time (Correct)  Average Response Time (Incorrect)  \
0                      2222.513514                             2671.0   
1                      2838.605263                             2807.5   
2                      2557.805556                             2568.5   

   Median Response Time  Median Response Time (Correct)  \
0                2492.0                          2492.0   
1                2604.0                          2604.0   
2                2456.5                          2456.5   

   Median Response Time (Incorrect)  
0                            2671.0  
1                            2807.5  
2                

In [139]:
def luScore():
    # grab test document
    lu = myUtil.findFile('LetterUpdating')

    # determine all unique trials
    trials = myUtil.getTrialType(lu)

    # subset data by trial
    trialDfs = myUtil.subsetByTrial(lu, trials)

    # create an empty list to store trial data
    trialData = []

    # score each trial (this will be general scores)
    for trial in trialDfs:
        trialName = trial.iloc[0, 4]
        pc = trial.iloc[:, 18].mean() / 3
        trialData.append([trialName, pc, trial.iloc[:, 18].count()])

    # create a pandas DataFrame to store trial data
    columnNames = ['Trial', 'Proportion Correct', 'Total Inputs']
    df = pd.DataFrame(trialData, columns=columnNames)

    # save the data frame to a CSV file
    df.to_csv('lu_scores.csv', float_format='%.2f', index=False)

    # output the DataFrame
    print(df)

luScore()

Successfully read file: 1001_LetterUpdating.txt
     Trial  Proportion Correct  Total Inputs
0  PRACT              0.666667             3
1  TRIAL1             0.875000             8
2  TRIAL2             0.666667             8


In [69]:
def msScore():
    
    # grab test document
    ms = myUtil.findFile('MotoricSpeed')

    # determine all unique trials
    trials = myUtil.getTrialType(ms, 5)

    # subset data by trial
    trialDfs = myUtil.subsetByTrial(ms, trials, 5)

    # score each trial (this will be general scores)
    for trial in trialDfs:
        print(f'\nScoring {trial.iloc[0, 5]}')
        pc = trial.iloc[:, 15].mean()
        print(f'Proportion correct: {pc:.2%} with total inputs: {trial.iloc[:, 13].count()}')
        responseTimes = trial.iloc[:, 14].mean()
        print(f'Average response time: {responseTimes:.2f}')
        responseCorrect = trial[trial.iloc[:, 15] == 1].iloc[:, 14].mean()
        print(f'Average response time for correct answers: {responseCorrect:.2f}')
        responseIncorrect = trial[trial.iloc[:, 15] == 0].iloc[:, 14].mean()
        print(f'Average response time for incorrect answers: {responseIncorrect:.2f}')
        medianResponseTime = trial.iloc[:, 14].median()
        print(f'Median response time: {medianResponseTime:.2f}')
        medianResponseCorrect = trial[trial.iloc[:, 15] == 1].iloc[:, 14].median()
        print(f'Median response time for correct answers: {medianResponseCorrect:.2f}')
        medianResponseIncorrect = trial[trial.iloc[:, 15] == 0].iloc[:, 14].median()
        print(f'Median response time for incorrect answers: {medianResponseIncorrect:.2f}')

msScore()

Successfully read file: 1001_MotoricSpeed.txt

Scoring PRACT1
Proportion correct: 59.09% with total inputs: 22
Average response time: 269.23
Average response time for correct answers: 423.69
Average response time for incorrect answers: 46.11
Median response time: 307.50
Median response time for correct answers: 354.00
Median response time for incorrect answers: 0.00

Scoring 5
Proportion correct: nan% with total inputs: 0
Average response time: nan
Average response time for correct answers: nan
Average response time for incorrect answers: nan
Median response time: nan
Median response time for correct answers: nan
Median response time for incorrect answers: nan

Scoring PRACT2
Proportion correct: 100.00% with total inputs: 24
Average response time: 318.88
Average response time for correct answers: 318.88
Average response time for incorrect answers: nan
Median response time: 303.00
Median response time for correct answers: 303.00
Median response time for incorrect answers: nan

Scoring T

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [70]:
def nmScore():
    
    # grab test document
    nm = myUtil.findFile('NumberMemory')

    # determine all unique trials
    trials = myUtil.getTrialType(nm, 4)

    # subset data by trial
    trialDfs = myUtil.subsetByTrial(nm, trials)

    # score each trial (this will be general scores)
    for trial in trialDfs:
        print(f'\nScoring {trial.iloc[0, 4]}')
        pc = trial.iloc[:, 10].mean()
        print(f'Proportion correct: {pc:.2%} with total inputs: {trial.iloc[:, 13].count()}')
        responseTimes = trial.iloc[:, 11].mean()
        print(f'Average response time: {responseTimes:.2f}')
        responseCorrect = trial[trial.iloc[:, 10] == 1].iloc[:, 11].mean()
        print(f'Average response time for correct answers: {responseCorrect:.2f}')
        responseIncorrect = trial[trial.iloc[:, 10] == 0].iloc[:, 11].mean()
        print(f'Average response time for incorrect answers: {responseIncorrect:.2f}')
        medianResponseTime = trial.iloc[:, 11].median()
        print(f'Median response time: {medianResponseTime:.2f}')
        medianResponseCorrect = trial[trial.iloc[:, 10] == 1].iloc[:, 11].median()
        print(f'Median response time for correct answers: {medianResponseCorrect:.2f}')
        medianResponseIncorrect = trial[trial.iloc[:, 10] == 0].iloc[:, 11].median()
        print(f'Median response time for incorrect answers: {medianResponseIncorrect:.2f}')

nmScore()

Successfully read file: 1001_NumberMemory.txt

Scoring PRACT1 
Proportion correct: 22.22% with total inputs: 9
Average response time: 4411.22
Average response time for correct answers: 3969.50
Average response time for incorrect answers: 4537.43
Median response time: 3594.00
Median response time for correct answers: 3969.50
Median response time for incorrect answers: 3252.00

Scoring TRIAL1 
Proportion correct: 20.00% with total inputs: 10
Average response time: 3808.90
Average response time for correct answers: 1567.00
Average response time for incorrect answers: 4369.38
Median response time: 3932.00
Median response time for correct answers: 1567.00
Median response time for incorrect answers: 4055.00

Scoring TRIAL2 
Proportion correct: 30.00% with total inputs: 10
Average response time: 2808.70
Average response time for correct answers: 2510.67
Average response time for incorrect answers: 2936.43
Median response time: 3013.00
Median response time for correct answers: 1788.00
Median r

In [72]:
# this test has a different trial format and the column is 6 instead of 5
def nbScore():
    
    # grab test document
    nb = myUtil.findFile('Numerical_nBack')

    # determine all unique trials
    trials = myUtil.getTrialType(nb, 5)
    print(trials)

    # subset data by trial
    trialDfs = myUtil.subsetByTrial(nb, trials, 5)

    # score each trial (this will be general scores)
    for trial in trialDfs:
        print(f'\nScoring {trial.iloc[0, 5]}')
        pc = trial.iloc[:, 13].mean()
        totalCorrect = trial.iloc[:, 13].sum()
        print(f'Proportion correct: {pc:.2%} (total correct = {totalCorrect}) with total inputs: {trial.iloc[:, 13].count()}')
        responseTimes = trial.iloc[:, 14].mean()
        print(f'Average response time: {responseTimes:.2f}')
        responseCorrect = trial[trial.iloc[:, 13] == 1].iloc[:, 14].mean()
        print(f'Average response time for correct answers: {responseCorrect:.2f}')
        responseIncorrect = trial[trial.iloc[:, 13] == 0].iloc[:, 14].mean()
        print(f'Average response time for incorrect answers: {responseIncorrect:.2f}')
        medianResponseTime = trial.iloc[:, 14].median()
        print(f'Median response time: {medianResponseTime:.2f}')
        medianResponseCorrect = trial[trial.iloc[:, 13] == 1].iloc[:, 14].median()
        print(f'Median response time for correct answers: {medianResponseCorrect:.2f}')
        medianResponseIncorrect = trial[trial.iloc[:, 13] == 0].iloc[:, 14].median()
        print(f'Median response time for incorrect answers: {medianResponseIncorrect:.2f}')

nbScore()

Successfully read file: 1001_Numerical_nBack.txt
['PRACT_1  ' 'PRACT_2  ' 'TRIAL1_1 ' 'TRIAL1_2 ' 'TRIAL2_1 ' 'TRIAL2_2 ']

Scoring PRACT_1  
Proportion correct: 58.62% (total correct = 17) with total inputs: 29
Average response time: 484.14
Average response time for correct answers: 825.88
Average response time for incorrect answers: 0.00
Median response time: 700.00
Median response time for correct answers: 785.00
Median response time for incorrect answers: 0.00

Scoring PRACT_2  
Proportion correct: 86.67% (total correct = 26) with total inputs: 30
Average response time: 672.67
Average response time for correct answers: 729.31
Average response time for incorrect answers: 304.50
Median response time: 679.00
Median response time for correct answers: 699.50
Median response time for incorrect answers: 0.00

Scoring TRIAL1_1 
Proportion correct: 86.67% (total correct = 26) with total inputs: 30
Average response time: 649.17
Average response time for correct answers: 749.04
Average respon

In [73]:
def nsScore():
    
    # grab test document
    ns = myUtil.findFile('NumericalSpeed')

    # determine all unique trials
    trials = myUtil.getTrialType(ns, 4)

    # subset data by trial
    trialDfs = myUtil.subsetByTrial(ns, trials)

    # score each trial (this will be general scores)
    for trial in trialDfs:
        print(f'\nScoring {trial.iloc[0, 4]}')
        pc = trial.iloc[:, 13].mean()
        print(f'Proportion correct: {pc:.2%} with total inputs: {trial.iloc[:, 13].count()}')
        responseTimes = trial.iloc[:, 14].mean()
        print(f'Average response time: {responseTimes:.2f}')
        responseCorrect = trial[trial.iloc[:, 13] == 1].iloc[:, 14].mean()
        print(f'Average response time for correct answers: {responseCorrect:.2f}')
        responseIncorrect = trial[trial.iloc[:, 13] == 0].iloc[:, 14].mean()
        print(f'Average response time for incorrect answers: {responseIncorrect:.2f}')
        medianResponseTime = trial.iloc[:, 14].median()
        print(f'Median response time: {medianResponseTime:.2f}')
        medianResponseCorrect = trial[trial.iloc[:, 13] == 1].iloc[:, 14].median()
        print(f'Median response time for correct answers: {medianResponseCorrect:.2f}')
        medianResponseIncorrect = trial[trial.iloc[:, 13] == 0].iloc[:, 14].median()
        print(f'Median response time for incorrect answers: {medianResponseIncorrect:.2f}')

nsScore()

Successfully read file: 1001_NumericalSpeed.txt

Scoring PRACT
Proportion correct: 94.87% with total inputs: 39
Average response time: 1036.67
Average response time for correct answers: 1021.03
Average response time for incorrect answers: 1326.00
Median response time: 993.00
Median response time for correct answers: 978.00
Median response time for incorrect answers: 1326.00

Scoring TRIAL1
Proportion correct: 95.00% with total inputs: 40
Average response time: 1050.78
Average response time for correct answers: 1062.18
Average response time for incorrect answers: 834.00
Median response time: 1014.00
Median response time for correct answers: 1026.00
Median response time for incorrect answers: 834.00

Scoring TRIAL2
Proportion correct: 100.00% with total inputs: 40
Average response time: 1031.97
Average response time for correct answers: 1031.97
Average response time for incorrect answers: nan
Median response time: 978.00
Median response time for correct answers: 978.00
Median response ti

In [74]:
def stScore():
    
    # grab test document
    st = myUtil.findFile('SpeedTabbing')

    # determine all unique trials
    trials = myUtil.getTrialType(st)

    # subset data by trial
    trialDfs = myUtil.subsetByTrial(st, trials)

    # score each trial (this will be general scores)
    for trial in trialDfs:
        print(f'\nScoring {trial.iloc[0, 4]}')
        numPresses = trial.iloc[:, 5].count()
        print(f'Total number of presses: {numPresses}')
        responseTimes = trial.iloc[:, 10].mean()
        print(f'Average response time: {responseTimes:.2f}')
        medianResponseTime = trial.iloc[:, 10].median()
        print(f'Median response time: {medianResponseTime:.2f}')
        firstPress = trial.iloc[0, 10]
        lastPress = trial.iloc[-1, 10]
        print(f'First press (ms): {firstPress:.2f} and last press (ms): {lastPress:.2f}')
        
stScore()

Successfully read file: 1001_SpeedTabbing.txt

Scoring PRACT1
Total number of presses: 158
Average response time: 123.00
Median response time: 120.00
First press (ms): 120.00 and last press (ms): 128.00

Scoring PRACT2
Total number of presses: 130
Average response time: 152.59
Median response time: 151.00
First press (ms): 253.00 and last press (ms): 127.00

Scoring TRIAL1
Total number of presses: 156
Average response time: 127.56
Median response time: 127.00
First press (ms): 77.00 and last press (ms): 119.00

Scoring TRIAL2
Total number of presses: 136
Average response time: 146.21
Median response time: 136.00
First press (ms): 87.00 and last press (ms): 167.00


In [75]:
def vsScore():
    
    # grab test document
    vs = myUtil.findFile('VerbalSpeed')

    # determine all unique trials
    trials = myUtil.getTrialType(vs)

    # subset data by trial
    trialDfs = myUtil.subsetByTrial(vs, trials)

    # score each trial (this will be general scores)
    for trial in trialDfs:
        print(f'\nScoring {trial.iloc[0, 4]}')
        pc = trial.iloc[:, 13].mean()
        print(f'Proportion correct: {pc:.2%} with total inputs: {trial.iloc[:, 13].count()}')
        responseTimes = trial.iloc[:, 14].mean()
        print(f'Average response time: {responseTimes:.2f}')
        responseCorrect = trial[trial.iloc[:, 13] == 1].iloc[:, 14].mean()
        print(f'Average response time for correct answers: {responseCorrect:.2f}')
        responseIncorrect = trial[trial.iloc[:, 13] == 0].iloc[:, 14].mean()
        print(f'Average response time for incorrect answers: {responseIncorrect:.2f}')
        medianResponseTime = trial.iloc[:, 14].median()
        print(f'Median response time: {medianResponseTime:.2f}')
        medianResponseCorrect = trial[trial.iloc[:, 13] == 1].iloc[:, 14].median()
        print(f'Median response time for correct answers: {medianResponseCorrect:.2f}')
        medianResponseIncorrect = trial[trial.iloc[:, 13] == 0].iloc[:, 14].median()
        print(f'Median response time for incorrect answers: {medianResponseIncorrect:.2f}')
        
vsScore()

Successfully read file: 1001_VerbalSpeed.txt

Scoring PRACT
Proportion correct: 100.00% with total inputs: 39
Average response time: 1053.38
Average response time for correct answers: 1053.38
Average response time for incorrect answers: nan
Median response time: 1026.00
Median response time for correct answers: 1026.00
Median response time for incorrect answers: nan

Scoring TRIAL1
Proportion correct: 97.50% with total inputs: 40
Average response time: 1035.05
Average response time for correct answers: 1038.38
Average response time for incorrect answers: 905.00
Median response time: 1014.50
Median response time for correct answers: 1034.00
Median response time for incorrect answers: 905.00

Scoring TRIAL2
Proportion correct: 100.00% with total inputs: 40
Average response time: 950.80
Average response time for correct answers: 950.80
Average response time for incorrect answers: nan
Median response time: 910.00
Median response time for correct answers: 910.00
Median response time for inc

In [103]:
def olmScore():

    # NOTE: consists of 6x6 grid of objects
    
    # grab test document
    olm = myUtil.findFile('ObjectLocationMemory')

    # determine all unique trials
    trials = myUtil.getTrialType(olm)
    print(trials)
    
    # subset data by trial
    trialDfs = myUtil.subsetByTrial(olm, trials)

    # score each trial (this will be general scores)
    for trial in trialDfs:
        print(f'\nScoring {trial.iloc[0, 4]}')
        pc = trial.iloc[:, 9].item()
        print(f'Proportion correct: {pc} of 12')
        responseTimes = trial.iloc[:, 7].item()
        print(f'Total response time: {responseTimes:.2f} ms')

    # Nitty gritty details
    # 1. Determine the original coordinates of the object
    # 2. Determine the coordinates of the object after the translation

olmScore()

Successfully read file: 1001_ObjectLocationMemory.txt
['TRIAL1' 'TRIAL2']

Scoring TRIAL1
Proportion correct: 7 of 12
Total response time: 159040.00 ms

Scoring TRIAL2
Proportion correct: 6 of 12
Total response time: 148840.00 ms


In [113]:
def suScore():
    # NOTE: consists of 3, 3x3 grids of objects
    
    # grab test document
    su = myUtil.findFile('SpatialUpdating')

    # determine all unique trials
    trials = myUtil.getTrialType(su)
    print(trials)

    # subset data by trial
    trialDfs = myUtil.subsetByTrial(su, trials)

    # score each trial (this will be general scores)
    for trial in trialDfs:
        print(f'\nScoring {trial.iloc[0, 4]}')
        pcGrid1 = trial.iloc[:, 33].mean()
        pcGrid2 = trial.iloc[:, 34].mean()
        pcGrid3 = trial.iloc[:, 35].mean()
        pcTotal = trial.iloc[:, 36].mean()
        print(f'Proportion correct for grid 1: {pcGrid1:.2%} with total inputs: {trial.iloc[:, 33].count()}')
        print(f'Proportion correct for grid 2: {pcGrid2:.2%} with total inputs: {trial.iloc[:, 34].count()}')
        print(f'Proportion correct for grid 3: {pcGrid3:.2%} with total inputs: {trial.iloc[:, 35].count()}')
        print(f'Proportion correct for all grids correct: {pcTotal:.2%} with total inputs: {trial.iloc[:, 36].count()}')
        

    # Nitty gritty details
    # 1. Determine the original coordinates of the object
    # 2. Determine the coordinates of the object after the translation
    
suScore()

Successfully read file: 1001_SpatialUpdating.txt
['PRACT' 'TRIAL1' 'TRIAL2']

Scoring PRACT
Proportion correct for grid 1: 100.00% with total inputs: 4
Proportion correct for grid 2: 25.00% with total inputs: 4
Proportion correct for grid 3: 50.00% with total inputs: 4
Proportion correct for all grids correct: 0.00% with total inputs: 4

Scoring TRIAL1
Proportion correct for grid 1: 80.00% with total inputs: 5
Proportion correct for grid 2: 40.00% with total inputs: 5
Proportion correct for grid 3: 20.00% with total inputs: 5
Proportion correct for all grids correct: 20.00% with total inputs: 5

Scoring TRIAL2
Proportion correct for grid 1: 40.00% with total inputs: 5
Proportion correct for grid 2: 40.00% with total inputs: 5
Proportion correct for grid 3: 40.00% with total inputs: 5
Proportion correct for all grids correct: 0.00% with total inputs: 5


In [121]:
def wrScore():
    
    # grab test document
    wr = myUtil.findFile('WordRecall')

    # determine all unique trials
    trials = myUtil.getTrialType(wr)

    # subset data by trial
    trialDfs = myUtil.subsetByTrial(wr, trials)

    # score each trial (this will be general scores)
    for trial in trialDfs:
        print(f'\nScoring {trial.iloc[0, 4]}')
        pc = trial.iloc[:, 24].item() / trial.iloc[:, 23].item()
        print(f'Proportion correct: {pc:.2%} with total responses: {trial.iloc[:, 23].item()}')
        intrusions = trial.iloc[:, 25].item() / trial.iloc[:, 23].item()
        print(f'Proportion of intrusions: {intrusions:.2%} with total responses: {trial.iloc[:, 23].item()}')
        

wrScore()

Successfully read file: 1001_WordRecall.txt

Scoring TRIAL1
Proportion correct: 100.00% with total responses: 11
Proportion of intrusions: 0.00% with total responses: 11

Scoring TRIAL2
Proportion correct: 88.89% with total responses: 9
Proportion of intrusions: 11.11% with total responses: 9


In [144]:
myUtil.makeScoredDir()

Successfully created directory: /Users/ethanjfranco/Desktop/GitHub/naftaliFiles/CognitiveTests/1001/scored
Successfully moved lu_scores.csv to /Users/ethanjfranco/Desktop/GitHub/naftaliFiles/CognitiveTests/1001/scored
Successfully moved fs_scores.csv to /Users/ethanjfranco/Desktop/GitHub/naftaliFiles/CognitiveTests/1001/scored
